In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from string import punctuation as en_punctuation
import re
from collections import Counter
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import issparse
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
periods_train_dir = r'/home/yw/study/Competition/Avito-Demand-Prediction-Challenge/data/periods_train.csv'
periods_test_dir = r'/home/yw/study/Competition/Avito-Demand-Prediction-Challenge/data/periods_test.csv'
train_active_dir = r'/home/yw/study/Competition/Avito-Demand-Prediction-Challenge/data/train_active.csv'
test_active_dir = r'/home/yw/study/Competition/Avito-Demand-Prediction-Challenge/data/test_active.csv'
train_dir = r'/home/yw/study/Competition/Avito-Demand-Prediction-Challenge/data/train.csv'
test_dir = r'/home/yw/study/Competition/Avito-Demand-Prediction-Challenge/data/test.csv'

In [4]:
train=pd.read_csv(train_dir)

In [6]:
periods_train = pd.read_csv(periods_train_dir)

In [36]:
periods_train.head(10)

,item_id,activation_date,date_from,date_to
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16
1,66218ff526d1,2017-02-16,2017-03-15,2017-03-18
2,b237d9539b21,2017-03-01,2017-03-15,2017-03-28
3,80bf58082ad3,2017-03-19,2017-03-19,2017-03-28
4,67a9944a7373,2017-03-14,2017-03-15,2017-03-28
5,47081d25b7cb,2017-02-19,2017-03-15,2017-03-20
6,5a328b4a8be1,2017-02-20,2017-03-15,2017-03-21
7,87292b76e4e1,2017-03-17,2017-03-17,2017-03-28
8,2ccefc8a940b,2017-02-19,2017-03-15,2017-03-21
9,ded64e61cd85,2017-03-13,2017-03-15,2017-03-28


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 18 columns):
item_id                 1503424 non-null object
user_id                 1503424 non-null object
region                  1503424 non-null object
city                    1503424 non-null object
parent_category_name    1503424 non-null object
category_name           1503424 non-null object
param_1                 1441848 non-null object
param_2                 848882 non-null object
param_3                 640859 non-null object
title                   1503424 non-null object
description             1387148 non-null object
price                   1418062 non-null float64
item_seq_number         1503424 non-null int64
activation_date         1503424 non-null object
user_type               1503424 non-null object
image                   1390836 non-null object
image_top_1             1390836 non-null float64
deal_probability        1503424 non-null float64
dtypes: float64(3), int

In [5]:
train.head(3)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


In [ ]:
# 针对训练集和测试集中每一个item_id,统计其激活日期当天同类别下正在展示的广告数量和均价，及该广告价格与均价的比值

In [ ]:
def stat_item_num_and_price(active_file):
    seed_data=pd.read_csv(seed_file)
    period_data=pd.read_csv(period_file)
    

In [38]:
train['activation_date'][0]>'2017-03-29'

False

In [30]:
agg_cat=['city','param_1','image_top_1','param_3','param_2','category_name','region','parent_category_name','activation_date']

In [31]:
train[agg_cat].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 8 columns):
city                    1503424 non-null object
param_1                 1441848 non-null object
image_top_1             1390836 non-null float64
param_3                 640859 non-null object
param_2                 848882 non-null object
category_name           1503424 non-null object
region                  1503424 non-null object
parent_category_name    1503424 non-null object
dtypes: float64(1), object(7)
memory usage: 91.8+ MB


In [32]:
train['title'].value_counts()

Платье                                                15550
Туфли                                                  6334
Куртка                                                 6331
Пальто                                                 5251
Джинсы                                                 4758
Комбинезон                                             4506
Кроссовки                                              3196
Костюм                                                 2916
Ботинки                                                2876
Босоножки                                              2760
Юбка                                                   2722
Кофта                                                  2721
Ветровка                                               2432
Сниму 1-к квартиру                                     2379
Сапоги                                                 2191
Свадебное платье                                       2183
Рубашка                                 

In [7]:
periods_train.head(3)

,item_id,activation_date,date_from,date_to
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16
1,66218ff526d1,2017-02-16,2017-03-15,2017-03-18
2,b237d9539b21,2017-03-01,2017-03-15,2017-03-28


In [23]:
ck_size=1200
ck=iter(pd.read_csv(train_active_dir,iterator = True,chunksize=ck_size))

In [24]:
row_num=0
while True:
    try:
        each = next(ck)
        row_num+=ck_size
    except StopIteration:
        break
print('row counts:',row_num)

row counts: 14130000
